# Data reading and cleaning

In [22]:
# import libraries and read csv
import pandas as pd
import numpy as np
import re
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [ ]:
# read csv
data = pd.read_csv(r"C:\Users\Vanessa\Downloads\Projeto Final\2016 - 2023.csv", usecols = [0, 1, 2, 4, 5, 7], index_col = None)

In [ ]:
# split csv into a list a dataframes
data_list = np.split(data, data[(data.isna().all(axis = 1))|(data.iloc[:,0].str.contains('MÊS'))].index)

In [ ]:
# identify not null dataframes from the list, rename columns and add them to another list
clean_data = []
for i in data_list:
    if i.isna().values.all():
        pass
    else:
        if i.iloc[:,0].str.contains('MÊS').any():
            i.columns = ['dia', 'receita', 'depositante', 'despesas', 'favorecido', 'saldo']
            clean_data.append(i)            

In [ ]:
# add "month" and "year" columns, reorder columns, drop unwanted rows and save dataframes as excel files
for i in range(len(clean_data)):
    clean_data[i]['mês'] = re.findall('\w+', clean_data[i].iloc[0,1])[0]
    clean_data[i]['ano'] = '20' + (re.findall('\w+', clean_data[i].iloc[0,1])[1]) 
    clean_data[i] = clean_data[i][['ano', 'mês', 'dia', 'receita', 'depositante', 'despesas', 'favorecido', 'saldo']]
    clean_data[i].drop(clean_data[i].index[0:3], inplace = True)
    clean_data[i].to_excel(f'tabela_{i}.xlsx', index = False)

In [13]:
# read excel files after manual cleaning and join tables in a single dataset
tables_df = pd.DataFrame()
for i in range(0,82):
    a = pd.read_excel(fr"C:\Users\Vanessa\Downloads\Dados\Programas\Repositório Git\NGO_DataAnalysis\clean tables\tabela_{i}.xls")
    tables_df = pd.concat([tables_df, a], ignore_index = True)

In [14]:
# create "expenses" dataframe and drop nulls
expenses = tables_df[['ano', 'mês', 'dia', 'despesas', 'favorecido']]
expenses.dropna(subset = 'despesas', axis = 0, inplace=True)

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_25424\2054822161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expenses.dropna(subset = 'despesas', axis = 0, inplace=True)


In [4]:
# create "income" dataframe and drop nulls
income = tables_df[['ano', 'mês', 'dia', 'receita', 'depositante']]
income.dropna(subset = 'receita', axis = 0, inplace=True)

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_25424\3992202683.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income.dropna(subset = 'receita', axis = 0, inplace=True)


In [15]:
# create categories for 'expenses'

mask = expenses['favorecido'].str.contains('sal|intercamb|alim|lanch|temperinho|rest|pão|café|panif|doce|confeitaria|subway|gratificação|sabor|hong ju|mansa', flags = re.I)
sal = np.where(mask, 'salário e auxílio alimentação', expenses['favorecido'])
expenses['favorecido'] = sal

mask2 = expenses['favorecido'].str.contains('sup|super|hiper|hipper|mercado|bistek|milium|embala|atacad|cassol|agua|distrib|loja|zeus|comerc', flags = re.I)
sup = np.where(mask2, 'suprimentos', expenses['favorecido'])
expenses['favorecido'] = sup

mask3 = expenses['favorecido'].str.contains('medic|clínica|clinica|vet|remédio|farm|biofilia|gral|asamed|pet|cuidados|panvel|acupuntura|raio x|praiana|castração|tosa|cremat', flags = re.I)
medic = np.where(mask3, 'medicações e veterinário', expenses['favorecido'])
expenses['favorecido'] = medic

mask4 = expenses['favorecido'].str.contains('hosp|poeta|lilian ribeiro|lar|cuidadora|casa|canil|canis|inse', flags = re.I)
hosp = np.where(mask4, 'hospedagem e manutenção de canis', expenses['favorecido'])
expenses['favorecido'] = hosp

mask5 = expenses['favorecido'].str.contains('agro|ração|reção|pecuária|terra|arcadia', flags = re.I)
rac = np.where(mask5, 'ração e agropecuária', expenses['favorecido'])
expenses['favorecido'] = rac

mask7 = expenses['favorecido'].str.contains('combust|escap|posto|seguro|veíc|motorista|car|borrack|park|auto|taxi|táxi|tàxi|frete|motor|estac|floripeças|multa|mecanica|mecânica|mecãnica|parachoque|borrach|estrela|localiza', flags = re.I)
trans = np.where(mask7, 'transporte', expenses['favorecido'])
expenses['favorecido'] = trans

mask6 = expenses['favorecido'].str.contains('camis|malhas|mat|jebelus|agenda|vest|patchwork|plumas|bazar|graf|gráf|digital|mtools|eletr|hd|site|notebook|face|cópias|copia|plot|mídia|chip|marca|mix|print', flags = re.I)
vend = np.where(mask6, 'bazar, divulgação e tecnologias', expenses['favorecido'])
expenses['favorecido'] = vend

mask8 = expenses['favorecido'].str.contains('empr|apli|renda', flags = re.I)
emp = np.where(mask8, 'empréstimos e aplicações', expenses['favorecido'])
expenses['favorecido'] = emp

mask9 = expenses['favorecido'].str.contains('tarif|INSS|IR|taxa|IPVA|DARF|celesc|tim|distrit|alvará|cadastro|licen|trib|certif|dss|cmf', flags = re.I)
tax = np.where(mask9, 'taxas, tarifas e impostos', expenses['favorecido'])
expenses['favorecido'] = tax

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_25424\1001768419.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expenses['favorecido'] = sal
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_25424\1001768419.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expenses['favorecido'] = sup
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_25424\1001768419.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [16]:
# create 'other' category for expenses 

outros_dict = {}
for item in expenses['favorecido']:
    if str(item) not in ['salário e auxílio alimentação', 'suprimentos', 'medicações e veterinário', 'hospedagem e manutenção de canis', 'ração e agropecuária', 'bazar, divulgação e tecnologias', 'transporte', 'empréstimos e aplicações', 'taxas, tarifas e impostos']:
        outros_dict[item] = True

others_mask = expenses['favorecido'].map(outros_dict)
others_mask = others_mask.fillna(False)
others = np.where(others_mask, 'outros', expenses['favorecido'])

expenses['favorecido'] = others

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_25424\2674992438.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expenses['favorecido'] = others


In [6]:
# create categories for income

mask = income['depositante'].str.contains('doaç|ração|v.d.|\d|DD|cielo|cielo|pag|pic', regex = True, flags = re.I)
doa = np.where(mask, 'doações e vendas', income['depositante'])
income['depositante'] = doa

mask2 = income['depositante'].str.contains('empr|apli|resg|rend', regex = True, flags = re.I)
emp = np.where(mask2, 'empréstimos e aplicações', income['depositante'])
income['depositante'] = emp

mask3 = income['depositante'].str.contains('Pista', regex = True, flags = re.I)
proj = np.where(mask3, 'projeto Auto Pista Litoral Sul', income['depositante'])
income['depositante'] = proj

mask4 = income['depositante'].str.contains('justiça|PMF ', regex = True, flags = re.I)
gov = np.where(mask4, 'governo', income['depositante'])
income['depositante'] = gov

mask5 = income['depositante'].str.contains('div|tran|int|seguro|rog|BB|dif', regex = True, flags = re.I)
out = np.where(mask5, 'outros', income['depositante'])
income['depositante'] = out

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_25424\1016977526.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income['depositante'] = doa
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_25424\1016977526.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income['depositante'] = emp
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_25424\1016977526.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [17]:
# final categories for 'expenses'
expenses['favorecido'].value_counts()

transporte                          1330
salário e auxílio alimentação       1025
taxas, tarifas e impostos            724
ração e agropecuária                 656
medicações e veterinário             536
outros                               321
hospedagem e manutenção de canis     244
suprimentos                          203
bazar, divulgação e tecnologias       82
Name: favorecido, dtype: int64

In [10]:
# final categories for 'income'
income['depositante'].value_counts()

doações e vendas                  1710
empréstimos e aplicações           411
projeto Auto Pista Litoral Sul      31
governo                             14
outros                              10
Name: depositante, dtype: int64

In [20]:
# read csv
animals = pd.read_excel(r"C:\Users\Vanessa\Downloads\Projeto Final\animais_2016_2019.xlsx", index_col = None)

In [38]:
# rename columns
animals.columns = animals.columns.str.lower().str.strip()

# Exploratory analysis

In [ ]:
# verificar número atual de animais

# Querying for metrics

In [40]:
# Custos totais para manter as atividades (2016-2023);
# Custos/ tipos de recurso (2016-2023);
# Média de recursos e custos por animal assistido (2016-2019);
# Total histórico de animais assistidos/aldeia (2016-2019.